In [ ]:
def f_plot_infogram(her):
    dim_cal = len(her['mat_euc_distance_xy'])
    # Delta_z_PMF x classes
    plt.figure()
    plt.plot(her['mat_euc_distance_xy'], her['mat_diff_z'], 'xb')
    plt.xlabel('Euclidean distance')
    plt.ylabel('\Deltaz')
    plt.title('Infogram cloud / cal. set: ' + str(dim_cal) + ' obs.')
    plt.gca().set_aspect('equal', adjustable='box')
    for i in range(1, len(her['edges_distance_classes'])):
        plt.axvline(x=her['edges_distance_classes'][i], color='red', linestyle='--')
    # Infogram
    hmax = np.max(her['mat_euc_distance_xy'])
    plt.figure()
    plt.plot(her['bin_centers_distance_classes'], her['H_diff_z_by_class'], marker='.', color='red', markersize=20)
    plt.title('Infogram / ' + her['txt'])
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axhline(y=her['H_diff_z'], linewidth=2)
    for i in range(1, len(her['edges_distance_classes'])):
        plt.axvline(x=her['edges_distance_classes'][i], color='black', linestyle='--')
    plt.legend(['Entropy of \Deltaz PMF by class', 'Entropy of the fullset \Deltaz PMF', 'Distance class limit'], loc='southwest')
    plt.xlabel('Euclidean distance')
    plt.ylabel('Entropy [bit]')
    # # of pairs/class
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax2.plot(range(1, her['n_lag']+1), her['H_diff_z_by_class'], marker='.', color='red', markersize=20)
    ax2.set_ylim([0, np.max(her['H_diff_z_by_class'])+np.max(her['H_diff_z_by_class'])/90])
    ax2.set_ylabel('Entropy [bit]')
    ax1.bar(range(1, her['n_lag']+1), her['n_pairs_by_class'])
    ax1.set_xticks([1] + list(range(5, her['n_lag']+1, 5)))
    ax1.set_xlabel('Distance class')
    ax1.set_ylabel('Number of pairs')
    ax1.set_title('Infogram & Number of pairs (in \Deltaz histogram) by class')
    plt.gca().set_aspect('equal', adjustable='box')
    str_ = ['# of observations: ' + str(dim_cal) + ' (cal. set)',
            '# of pairs: ' + str(np.sum(her['n_pairs_by_class'])) + ' (' + str(dim_cal) + 'x' + str(dim_cal-1) + ')',
            '# of pairs into the range: ' + str(np.sum(her['n_pairs_by_class'][:her['n_classes_range']-1])) + ' (' + str(round(100*np.sum(her['n_pairs_by_class'][:her['n_classes_range']-1])/np.sum(her['n_pairs_by_class']), 1)) + '%)']
    plt.text(her['n_lag']/1.3, np.max(her['n_pairs_by_class'])/1, str_)
    # delta-z PMF/class
    ncols = 5
    nrows = int(np.ceil(((len(her['bin_centers_distance_classes_range'])) / ncols)))
    plt.figure()
    for i in range(1, her['n_classes_range']+1):
        plt.subplot(nrows, ncols, i)
        plt.bar(her['bin_centers_edges_diff_z'], her['pmf_diff_z_by_class_obs_range'][i-1, :], 0.9)
        plt.title('class ' + str(i) + ' / dist.: (' + str(round(her['edges_distance_classes_range'][i-1], 2)) + ', ' + str(int(her['edges_distance_classes_range'][i])) + ']')
        plt.gca().set_aspect('equal', adjustable='box')
        plt.xlabel('\Deltaz')
        plt.ylim([0, np.max(her['pmf_diff_z_by_class_obs_range'])+np.max(her['pmf_diff_z_by_class_obs_range'])/50])
        plt.xlim([-5, 5])
    plt.title('class ' + str(i) + ' (full dataset PMF)')
    plt.suptitle('\Deltaz PMF by class')
    # EXTRA step: Variogram
    lag_variance = np.full(len(her['edges_distance_classes'])-1, np.nan) #vector for the semivariance of the observations
    n_obs_by_lag = np.zeros(len(her['edges_distance_classes'])-1) #vector for counting the number of pair points in each lag class
    mat_square_diff_z = her['mat_diff_z']**2 #matrix of the squared semivariance
    for i in range(len(her['edges_distance_classes'])-1): #for each distance class
        idx = np.logical_and(her['mat_euc_distance_xy'] > her['edges_distance_classes'][i], her['mat_euc_distance_xy'] <= her['edges_distance_classes'][i+1]) #find the observations within the current lag class
        n_obs_by_lag[i] = np.sum(idx) #count the number of values
        lag_variance[i] = 1/2 * np.mean(mat_square_diff_z[idx]) #calculate the semivariance
    #Infogram + Variogram
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(her['bin_centers_distance_classes'], her['H_diff_z_by_class']/np.max(her['H_diff_z_by_class']), marker='.', markersize=20)
    ax1.set_ylabel('Normalized entropy [bit]')
    ax1.set_xlim([0, her['bin_centers_distance_classes'][-2]])
    ax1.set_ylim([0, 1])
    ax2.plot(her['bin_centers_distance_classes'], lag_variance/np.max(lag_variance), marker='.', markersize=20)
    ax2.set_xlabel('lag')
    ax2.set_ylabel('Normalized dissimilarity (1/2*mean(\Deltaz²))')
    ax2.set_ylim([0, 1])
    for i in range(1, len(her['edges_distance_classes'])):
        ax1.axvline(x=her['edges_distance_classes'][i], color='black', linestyle='--')
    plt.title('Normalized Infogram and Experimental variogram')